In [1]:
# Load environment var
import os
from dotenv import load_dotenv

load_dotenv()

True

In [2]:
# Access environment vars
os.environ['OPENAI_API_KEY'] = os.getenv('OPENAI_API_KEY')

os.environ['LANGCHAIN_API_KEY'] = os.getenv('LANGCHAIN_API_KEY')
os.environ['LANGCHAIN_TRACING_V2'] = os.getenv('LANGCHAIN_TRACING_V2')
os.environ['LANGCHAIN_PROJECT'] = os.getenv('LANGCHAIN_PROJECT')

In [3]:
from langchain_openai import ChatOpenAI

model = ChatOpenAI(model='gpt-4o-mini')
model

ChatOpenAI(client=<openai.resources.chat.completions.completions.Completions object at 0x00000272B83EEA50>, async_client=<openai.resources.chat.completions.completions.AsyncCompletions object at 0x00000272B83EF4D0>, root_client=<openai.OpenAI object at 0x00000272B83EC050>, root_async_client=<openai.AsyncOpenAI object at 0x00000272B83EF230>, model_name='gpt-4o-mini', model_kwargs={}, openai_api_key=SecretStr('**********'))

In [4]:
model.invoke('What is generative ai?')

AIMessage(content='Generative AI refers to a class of artificial intelligence models and algorithms that can create new content, such as text, images, music, and other forms of media, by learning patterns from existing data. Unlike traditional AI, which is typically focused on recognizing patterns or making predictions based on input data, generative AI aims to generate new, original outputs that mimic the characteristics of the training data.\n\nThere are several key components and techniques associated with generative AI:\n\n1. **Models**: Common models used in generative AI include Generative Adversarial Networks (GANs), Variational Autoencoders (VAEs), and more recently, transformer-based architectures like GPT (Generative Pre-trained Transformer) for text generation and DALL-E for image generation.\n\n2. **Training**: Generative AI models are trained on large datasets to learn the underlying structures and distributions of the data. For instance, a text-based model might be traine

### Prompt

In [5]:
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_messages(
    [
        ('system', 'You are an export AI Engineer. Provide an answer based on the question.'),
        ('user', '{input_text}')
    ]
)

prompt

ChatPromptTemplate(input_variables=['input_text'], input_types={}, partial_variables={}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='You are an export AI Engineer. Provide an answer based on the question.'), additional_kwargs={}), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input_text'], input_types={}, partial_variables={}, template='{input_text}'), additional_kwargs={})])

In [6]:
chain = prompt | model
response = chain.invoke({ 'input_text': 'What is agentic ai?'})
response

AIMessage(content='Agentic AI refers to artificial intelligence systems that possess a degree of autonomy and decision-making capability, allowing them to act independently and take initiative in completing tasks without constant human intervention. This often involves the ability to understand and adapt to complex environments, make choices based on predefined goals or learned experiences, and potentially interact with other systems or agents.\n\nKey characteristics of agentic AI can include:\n\n1. **Autonomy**: The ability to operate without human oversight, making decisions and taking actions on its own.\n  \n2. **Goal-oriented behavior**: Agentic AI is designed to achieve specific objectives, which may involve planning, reasoning, and learning from the environment.\n\n3. **Adaptability**: The capacity to adjust actions based on changing circumstances or new information.\n\n4. **Interactivity**: The ability to engage with human users or other systems, potentially collaborating or ne

### Output Parser

In [7]:
from langchain_core.output_parsers import StrOutputParser

output_parser = StrOutputParser()

chain = prompt | model | output_parser
response = chain.invoke({ 'input_text': 'What is agentic ai?'})
print(response)

Agentic AI refers to artificial intelligence systems that possess a degree of autonomy and decision-making capabilities, allowing them to operate independently to achieve specific goals or tasks. Unlike traditional AI, which may follow predefined rules or scripts, agentic AI can analyze its environment, adapt to new information, and make decisions based on its understanding of the context.

Key features of agentic AI include:

1. **Autonomy**: The ability to operate without human intervention, making choices based on its programming and learning from its experiences.

2. **Goal-oriented behavior**: Designed to pursue specific objectives and optimize outcomes based on those goals.

3. **Learning and Adaptation**: Incorporates machine learning techniques to improve its performance over time, learning from interactions and feedback.

4. **Interaction with the Environment**: Capable of perceiving its environment, collecting data, and responding to changes in real time.

5. **Decision-makin

In [8]:
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import JsonOutputParser

json_output_parser = JsonOutputParser()

prompt = PromptTemplate(
    template="Answer the user query in \n{format_instruction}\n{query}\n",
    input_variables=["query"],
    partial_variables={"format_instruction": json_output_parser.get_format_instructions()}
)

prompt

PromptTemplate(input_variables=['query'], input_types={}, partial_variables={'format_instruction': 'Return a JSON object.'}, template='Answer the user query in \n{format_instruction}\n{query}\n')

In [9]:
chain = prompt | model | json_output_parser

response = chain.invoke({"query":"Can you tell me about Langsmith?"})
print(response)

{'name': 'Langsmith', 'description': 'Langsmith is an innovative platform designed to enhance the development and integration of language models. It provides tools and resources for creating, testing, and deploying AI-driven language processing applications.', 'features': ['User-friendly interface for model training and management', 'Integration capabilities with existing workflows', 'Support for various programming languages and frameworks', 'Collaboration tools for teams working on language model projects', 'In-depth analytics and performance tracking'], 'use_cases': ['Creating chatbots and virtual assistants', 'Automating content generation and language translation', 'Sentiment analysis and customer feedback processing', 'Improving user experience through personalized interactions'], 'website': 'https://www.langsmith.com', 'launch_year': 2023, 'target_audience': ['Developers', 'Data scientists', 'Businesses looking to leverage AI in their operations', 'Researchers in natural languag

In [11]:
chain = prompt | model | output_parser
response = chain.invoke("Steps to implement Agentic RAG.")
print(response)

```json
{
  "steps_to_implement_agentic_RAG": [
    {
      "step": 1,
      "description": "Define the objective of the RAG system.",
      "details": "Identify the specific tasks or outcomes that the RAG system will aim to achieve, such as retrieving relevant documents or generating responses."
    },
    {
      "step": 2,
      "description": "Collect and preprocess data.",
      "details": "Gather relevant datasets that will be used for retrieval. Preprocess the data to ensure it is clean and structured for efficient retrieval."
    },
    {
      "step": 3,
      "description": "Develop retrieval mechanism.",
      "details": "Implement a search or retrieval engine that can efficiently query the dataset and return relevant information based on user requests."
    },
    {
      "step": 4,
      "description": "Integrate with a language model.",
      "details": "Connect the retrieval mechanism to a language model (such as GPT) that will be responsible for generating responses bas

### RAG

In [10]:
from langchain_community.document_loaders import WebBaseLoader

# load data
loader = WebBaseLoader("https://python.langchain.com/docs/tutorials/llm_chain/")
documents = loader.load()
documents

[Document(metadata={'source': 'https://python.langchain.com/docs/tutorials/llm_chain/', 'title': 'Build a simple LLM application with chat models and prompt templates | 🦜️🔗 LangChain', 'description': "In this quickstart we'll show you how to build a simple LLM application with LangChain. This application will translate text from English into another language. This is a relatively simple LLM application - it's just a single LLM call plus some prompting. Still, this is a great way to get started with LangChain - a lot of features can be built with just some prompting and an LLM call!", 'language': 'en'}, page_content='\n\n\n\n\nBuild a simple LLM application with chat models and prompt templates | 🦜️🔗 LangChain\n\n\n\n\n\n\nSkip to main contentOur Building Ambient Agents with LangGraph course is now available on LangChain Academy!IntegrationsAPI ReferenceMoreContributingPeopleError referenceLangSmithLangGraphLangChain HubLangChain JS/TSv0.3v0.3v0.2v0.1💬SearchIntroductionTutorialsBuild a 

In [21]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

# splitting the text
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)
final_documents = text_splitter.split_documents(documents)
final_documents

[Document(metadata={'source': 'https://python.langchain.com/docs/tutorials/llm_chain/', 'title': 'Build a simple LLM application with chat models and prompt templates | 🦜️🔗 LangChain', 'description': "In this quickstart we'll show you how to build a simple LLM application with LangChain. This application will translate text from English into another language. This is a relatively simple LLM application - it's just a single LLM call plus some prompting. Still, this is a great way to get started with LangChain - a lot of features can be built with just some prompting and an LLM call!", 'language': 'en'}, page_content='Build a simple LLM application with chat models and prompt templates | 🦜️🔗 LangChain'),
 Document(metadata={'source': 'https://python.langchain.com/docs/tutorials/llm_chain/', 'title': 'Build a simple LLM application with chat models and prompt templates | 🦜️🔗 LangChain', 'description': "In this quickstart we'll show you how to build a simple LLM application with LangChain.

In [20]:
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings()
embeddings

OpenAIEmbeddings(client=<openai.resources.embeddings.Embeddings object at 0x000001B8901682D0>, async_client=<openai.resources.embeddings.AsyncEmbeddings object at 0x000001B890168B90>, model='text-embedding-ada-002', dimensions=None, deployment='text-embedding-ada-002', openai_api_version=None, openai_api_base=None, openai_api_type=None, openai_proxy=None, embedding_ctx_length=8191, openai_api_key=SecretStr('**********'), openai_organization=None, allowed_special=None, disallowed_special=None, chunk_size=1000, max_retries=2, request_timeout=None, headers=None, tiktoken_enabled=True, tiktoken_model_name=None, show_progress_bar=False, model_kwargs={}, skip_empty=False, default_headers=None, default_query=None, retry_min_seconds=4, retry_max_seconds=20, http_client=None, http_async_client=None, check_embedding_ctx_length=True)

In [22]:
from langchain_community.vectorstores import FAISS

vectorstore = FAISS.from_documents(documents=final_documents, embedding=embeddings)
vectorstore

In [24]:
query = "Which application will translate text from English into another language"

result = vectorstore.similarity_search(query)
result[0].page_content

'language: The language to translate text into\ntext: The text to translate'

In [28]:
query = "What is LangSmith?"

result = vectorstore.similarity_search(query)
print(result[0].page_content)

Chat models
Prompt templates

And the LangSmith docs:

LangSmith
Edit this pagePreviousTutorialsNextBuild a ChatbotSetupJupyter NotebookInstallationLangSmithUsing Language ModelsStreamingPrompt TemplatesConclusionCommunityLangChain ForumTwitterGitHubOrganizationPythonJS/TSMoreHomepageBlogYouTubeCopyright © 2025 LangChain, Inc.
